In [6]:
  return (
    <div className={styles.container}>
      <div className={styles.actionBar}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>Email Actions</Text>
          </div>
          <Text className={styles.actionDescription}>Select an action to perform on your email.</Text>
          {selectedAction && selectedAction.requiresInput && (
            <>
              <Text className={styles.actionDescription}>{selectedAction.description}</Text>
              <textarea
                className={styles.input}
                placeholder="Enter any additional instructions here..."
                value={inputValue}
                onChange={(e) => setInputValue(e.target.value)}
              />
            </>
          )}
          <div className={styles.buttonContainer}>
            {buttonConfig.map((action, index) => (
              <Button
                key={index}
                appearance={selectedAction === action ? "primary" : "secondary"}
                onClick={() => handleButtonClick(action)}
                disabled={isProcessing || (selectedAction && selectedAction !== action)}
              >
                {action.label}
              </Button>
            ))}
            {selectedAction && selectedAction.requiresInput && (
              <Button
                appearance="primary"
                onClick={() => handleAction(selectedAction, inputValue)}
                disabled={isProcessing || !inputValue.trim()}
              >
                Submit
              </Button>
            )}
          </div>
        </div>
      </div>
      <div className={styles.statusContainer}>
        {isProcessing && (
          <div className={styles.statusMessage}>
            <Spinner size="tiny" />
            <span style={{ marginLeft: '8px' }}>Processing your request...</span>
          </div>
        )}
        {statusMessage && (
          <div className={`${styles.statusMessage} ${styles.successMessage}`}>
            <div>{statusMessage.text}</div>
          </div>
        )}
        {error && (
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        )}
      </div>
      {generatedResponse && (
        <div className={styles.responseContainer}>
          <Text className={styles.responseText}>{generatedResponse}</Text>
        </div>
      )}
    </div>
  );
};

export default EmailGenerator;

ModuleNotFoundError: No module named 'streamlit'